In [283]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import mplfinance as mpf

In [3]:
df = pd.read_csv('15mins_test.csv')

In [6]:
df.drop('Unnamed: 0',axis = 1,inplace = True)

In [40]:
def Pivots_Camarilla(df_OHLC,timeframe):
    df_OHLC = df_OHLC.sort_values(['SYMBOL','TS1']).groupby('SYMBOL').tail(150).reset_index()
    df_OHLC['TIMEFRAME'] = timeframe
    df_OHLC['P'] = (df_OHLC.HIGH + df_OHLC.LOW + df_OHLC.CLOSE ) / 3.0
    df_OHLC['RANGE'] = df_OHLC.HIGH - df_OHLC.LOW
    df_OHLC["H1"] = df_OHLC.CLOSE + (df_OHLC.HIGH - df_OHLC.LOW) * 1.1 / 12.0
    df_OHLC["H2"] = df_OHLC.CLOSE + (df_OHLC.HIGH - df_OHLC.LOW) * 1.1 / 6.0
    df_OHLC["H3"] = df_OHLC.CLOSE + (df_OHLC.HIGH - df_OHLC.LOW) * 1.1 / 4.0
    df_OHLC["H4"] = df_OHLC.CLOSE + (df_OHLC.HIGH - df_OHLC.LOW) * 1.1 / 2.0
    df_OHLC["H5"] = (df_OHLC.HIGH/df_OHLC.LOW) * df_OHLC.CLOSE
    df_OHLC["H6"] = df_OHLC.H5 + 1.168 * (df_OHLC.H5 - df_OHLC.H4)
    df_OHLC["L1"] = df_OHLC.CLOSE - (df_OHLC.HIGH - df_OHLC.LOW) * 1.1 / 12.0
    df_OHLC["L2"] = df_OHLC.CLOSE - (df_OHLC.HIGH - df_OHLC.LOW) * 1.1 / 6.0
    df_OHLC["L3"] = df_OHLC.CLOSE - (df_OHLC.HIGH - df_OHLC.LOW) * 1.1 / 4.0
    df_OHLC["L4"] = df_OHLC.CLOSE - (df_OHLC.HIGH - df_OHLC.LOW) * 1.1 / 2.0
    df_OHLC["L5"] = df_OHLC.CLOSE - (df_OHLC.H5 - df_OHLC.CLOSE)
    df_OHLC["L6"] = df_OHLC.CLOSE - (df_OHLC.H6 - df_OHLC.CLOSE)
    df_OHLC["BC"] = (df_OHLC.HIGH + df_OHLC.LOW ) / 2.0
    df_OHLC["TC"] = (df_OHLC.P - df_OHLC.BC) + df_OHLC.P
    df_OHLC["VAH"] = df_OHLC.H3
    df_OHLC["VAL"] = df_OHLC.L3
    df_OHLC["VA"] = df_OHLC.VAH - df_OHLC.VAL
    df_OHLC.BC,df_OHLC.TC = np.where(df_OHLC.TC < df_OHLC.BC,(df_OHLC.TC,df_OHLC.BC),(df_OHLC.BC,df_OHLC.TC))
    df_OHLC['CPR'] = abs(df_OHLC.TC - df_OHLC.BC)*100/df_OHLC.P
    df_OHLC = df_OHLC.sort_values(['SYMBOL','TS1'])
    LAST_6_DAYS_CPR_RANGE_WIDE = df_OHLC.groupby(['SYMBOL']).apply(lambda x : pd.Series({
        'CPR_WIDE_CONFORMATION' : 1 if (max(x.CPR.values[-6:-1])) < (x.CPR.values[-1]) else 0 })).reset_index()
    LAST_6_DAYS_CPR_RANGE_NARROW = df_OHLC.groupby(['SYMBOL']).apply(lambda x : pd.Series({
        'CPR_NARROW_CONFORMATION' : 1 if (min(x.CPR.values[-6:-1])) > (x.CPR.values[-1]) else 0 })).reset_index()
    df_OHLC = pd.merge(df_OHLC,LAST_6_DAYS_CPR_RANGE_WIDE,left_on='SYMBOL',right_on='SYMBOL')
    df_OHLC = pd.merge(df_OHLC,LAST_6_DAYS_CPR_RANGE_NARROW,left_on='SYMBOL',right_on='SYMBOL')
    cpr_narrow = df_OHLC.groupby(['SYMBOL']).CPR.quantile(q=0.15)
    cpr_wide = df_OHLC.groupby(['SYMBOL']).CPR.quantile(q=0.85)
    df_avg_cpr_va = cpr_narrow.to_frame(name="CPR_LOW").reset_index().merge(
        cpr_wide.to_frame(name="CPR_UP").reset_index(),
        left_on="SYMBOL",
        right_on="SYMBOL")
    df_OHLC = df_OHLC.merge(df_avg_cpr_va,
                  left_on="SYMBOL",
                  right_on="SYMBOL")

    va_narrow = df_OHLC.groupby(['SYMBOL']).VA.quantile(q=0.35)
    va_wide = df_OHLC.groupby(['SYMBOL']).VA.quantile(q=0.65)
    df_avg_va = va_narrow.to_frame(name="VA_LOW").reset_index().merge(
        va_wide.to_frame(name="VA_UP").reset_index(),
        left_on="SYMBOL",
        right_on="SYMBOL")
    df_OHLC = df_OHLC.merge(df_avg_va,
                  left_on="SYMBOL",
                  right_on="SYMBOL")

    df_OHLC = df_OHLC
    df_OHLC = df_OHLC.sort_values(['SYMBOL','TS1'])
    df_OHLC['CPR_WIDE'] = np.where(df_OHLC.CPR_WIDE_CONFORMATION == 1,(np.where(df_OHLC.CPR >= df_OHLC.CPR_UP,1,0)),0)
    df_OHLC['CPR_NARROW'] = np.where(df_OHLC.CPR_NARROW_CONFORMATION == 1,np.where(df_OHLC.CPR <= df_OHLC.CPR_LOW,1,0),0)
    df_OHLC = df_OHLC.round(2).copy()#[['SYMBOL','TS1','BC','P','TC','OPEN','CLOSE','H1','H2','H3','H4','H5','H6','L1','L2','L3','L4','L5','L6','VAH','VAL','VA']]
    df_OHLC['OPEN_ABOVE_H3'] = np.where(((df_OHLC.OPEN > df_OHLC.H3) & (df_OHLC.OPEN < df_OHLC.H4)),1,0)
    df_OHLC['OPEN_ABOVE_H4'] = np.where((df_OHLC.OPEN > df_OHLC.H4),1,0)
    df_OHLC['OPEN_BELOW_L3'] = np.where(((df_OHLC.OPEN < df_OHLC.L3) & (df_OHLC.OPEN > df_OHLC.L4)),1,0)
    df_OHLC['OPEN_BELOW_L4'] = np.where((df_OHLC.OPEN < df_OHLC.L4),1,0)
    df_OHLC['PREV_BC'] = df_OHLC.groupby(['SYMBOL']).BC.shift(1)
    df_OHLC['PREV_P'] = df_OHLC.groupby(['SYMBOL']).P.shift(1)
    df_OHLC['PREV_TC'] = df_OHLC.groupby(['SYMBOL']).TC.shift(1)
    df_OHLC['BULLISH'] = np.where((df_OHLC.BC > df_OHLC.PREV_TC),1,0)
    df_OHLC['BEARISH'] = np.where((df_OHLC.TC < df_OHLC.PREV_BC),1,0)
    df_OHLC['MODERATELY_BULLISH'] = np.where(((df_OHLC.TC > df_OHLC.PREV_TC) & ((df_OHLC.BC < df_OHLC.PREV_TC) & (df_OHLC.BC > df_OHLC.PREV_BC))),1,0)#Current TC is Higher thatn previous TC but overlapping
    df_OHLC['MODERATELY_BEARISH'] = np.where(((df_OHLC.BC < df_OHLC.PREV_BC) & ((df_OHLC.TC > df_OHLC.PREV_BC) & (df_OHLC.TC < df_OHLC.PREV_TC)) ),1,0) #Current BC is less thatn previous BC but overlappin
    df_OHLC['NEUTRAL'] = np.where(((df_OHLC.BC == df_OHLC.PREV_BC) & (df_OHLC.TC == df_OHLC.PREV_TC)),1,0)
    df_OHLC['SIDEWAYS'] = np.where(((df_OHLC.BC < df_OHLC.PREV_BC) & (df_OHLC.TC > df_OHLC.PREV_TC)),1,0) ##ENGULFING
    df_OHLC['BREAKOUT'] = np.where((((df_OHLC.BC > df_OHLC.PREV_BC) & (df_OHLC.TC < df_OHLC.PREV_P)) |
                               ((df_OHLC.BC > df_OHLC.PREV_P) & (df_OHLC.TC < df_OHLC.PREV_TC))
                              ),1,0)
    df_OHLC['PREVIOUS_H3'] = df_OHLC.groupby(['SYMBOL']).H3.shift(1)
    df_OHLC = df_OHLC.dropna()
    df_OHLC_2 = df_OHLC[['SYMBOL','BULLISH','BEARISH']].copy()
    # df_OHLC_2['STATUS'] = 0
    df_OHLC_2['CURRENT_STATUS'] = np.where((df_OHLC_2.BULLISH == 1),1,np.where(df_OHLC_2.BEARISH == 1,-1,0))
    df_OHLC_2['PREVIOUS_STATUS'] = 0
    df_OHLC_2['PREVIOUS_STATUS'] = df_OHLC_2.groupby(['SYMBOL']).CURRENT_STATUS.shift(1)
    df_OHLC_2.dropna(inplace=True)
    df_OHLC_2['PREVIOUS_STATUS'] = df_OHLC_2.PREVIOUS_STATUS.astype(int)
    df_OHLC_2['STATUS'] = np.where((df_OHLC_2.CURRENT_STATUS == df_OHLC_2.PREVIOUS_STATUS),0,df_OHLC_2.CURRENT_STATUS)
    df_OHLC_2 = df_OHLC_2.groupby(['SYMBOL']).tail(50)
    df_OHLC_2['SCORE']=0
    df_OHLC_2 = df_OHLC_2.groupby(['SYMBOL']).apply(lambda x :
                                 pd.DataFrame({
                                     'SYMBOL' : x.SYMBOL,
                                     'SCORE' : np.where(x.STATUS.values[::-1].argmax() < (x.STATUS.values[::-1].argmin()),
                                              1*(x.STATUS.values[::-1].argmin()-x.STATUS.values[-x.STATUS.values[::-1].argmin():].argmax()),
                                              -1*(x.STATUS.values[::-1].argmax()-x.STATUS.values[-x.STATUS.values[::-1].argmax():].argmin()))
                                 })
                                ).groupby(['SYMBOL']).last().reset_index()
    df_OHLC = pd.merge(df_OHLC,df_OHLC_2,how='left',on='SYMBOL')
    df_OHLC = df_OHLC[["SYMBOL","TS1","P","BC","TC",
                       "TIMEFRAME",'CPR_WIDE','CPR_NARROW',
                       "L1","L2","L3","L4","L5","L6",
                       "H1","H2","H3","H4","H5",'H6',
                       "BULLISH","BEARISH","MODERATELY_BULLISH",
                       "MODERATELY_BEARISH","NEUTRAL",
                       "SIDEWAYS","BREAKOUT",'SCORE']].groupby('SYMBOL').last().reset_index()

    return df_OHLC

In [36]:
df = df.rename(columns=str.upper)
df.TS1 = pd.to_datetime(df.TS1)
df.TS1 = df.TS1.dt.date

In [72]:
df_new = Pivots_Camarilla(df,'DAY')

In [74]:
df_new.head()

,SYMBOL,TS1,P,BC,TC,TIMEFRAME,CPR_WIDE,CPR_NARROW,L1,L2,...,H5,H6,BULLISH,BEARISH,MODERATELY_BULLISH,MODERATELY_BEARISH,NEUTRAL,SIDEWAYS,BREAKOUT,SCORE
0,ACC,2021-01-01,1627.62,1626.81,1628.42,DAY,0,0,1625.53,1625.06,...,1631.15,1633.86,1,0,0,0,0,0,0,2
1,ADANIENT,2021-01-01,489.80,489.40,490.20,DAY,0,0,488.71,488.41,...,492.20,493.89,0,1,0,0,0,0,0,-1
2,ADANIPORTS,2021-01-01,505.15,505.08,505.22,DAY,0,0,504.52,504.04,...,510.28,513.06,1,0,0,0,0,0,0,3
3,AMARAJABAT,2021-01-01,923.73,923.62,923.84,DAY,0,0,923.57,923.19,...,928.11,930.30,0,1,0,0,0,0,0,-2
4,AMBUJACEM,2021-01-01,250.95,250.95,250.95,DAY,0,1,250.81,250.68,...,252.45,253.25,1,0,0,0,0,0,0,2


In [82]:
df_new

,SYMBOL,TS1,P,BC,TC,TIMEFRAME,CPR_WIDE,CPR_NARROW,L1,L2,L3,L4,L5,L6,H1,H2,H3,H4,H5,H6,BULLISH,BEARISH,MODERATELY_BULLISH,MODERATELY_BEARISH,NEUTRAL,SIDEWAYS,BREAKOUT,SCORE,CPR
0,ACC,2021-01-01,1627.62,1626.81,1628.42,DAY,0,0,1625.53,1625.06,1624.58,1623.17,1620.85,1618.14,1626.47,1626.94,1627.42,1628.83,1631.15,1633.86,1,0,0,0,0,0,0,2,0.098917
1,ADANIENT,2021-01-01,489.80,489.40,490.20,DAY,0,0,488.71,488.41,488.12,487.24,485.80,484.11,489.29,489.59,489.88,490.76,492.20,493.89,0,1,0,0,0,0,0,-1,0.163332
2,ADANIPORTS,2021-01-01,505.15,505.08,505.22,DAY,0,0,504.52,504.04,503.56,502.11,499.72,496.94,505.48,505.96,506.44,507.89,510.28,513.06,1,0,0,0,0,0,0,3,0.027715
3,AMARAJABAT,2021-01-01,923.73,923.62,923.84,DAY,0,0,923.57,923.19,922.81,921.67,919.79,917.60,924.33,924.71,925.09,926.23,928.11,930.30,0,1,0,0,0,0,0,-2,0.023816
4,AMBUJACEM,2021-01-01,250.95,250.95,250.95,DAY,0,1,250.81,250.68,250.54,250.12,249.45,248.65,251.09,251.22,251.36,251.77,252.45,253.25,1,0,0,0,0,0,0,2,0.000000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
133,UPL,2021-01-01,468.77,468.45,469.08,DAY,1,0,469.12,468.83,468.55,467.69,466.28,464.63,469.68,469.97,470.25,471.10,472.52,474.17,0,1,0,0,0,0,0,-3,0.134394
134,VEDL,2021-01-01,160.40,160.30,160.50,DAY,0,0,160.54,160.49,160.44,160.27,160.00,159.68,160.66,160.71,160.76,160.93,161.20,161.52,0,0,0,0,0,0,0,-5,0.124688
135,VOLTAS,2021-01-01,831.75,831.62,831.88,DAY,0,0,831.33,831.16,830.99,830.48,829.65,828.67,831.67,831.84,832.01,832.52,833.35,834.33,1,0,0,0,0,0,0,2,0.031259
136,WIPRO,2021-01-01,387.87,387.76,387.98,DAY,0,0,387.55,387.46,387.36,387.07,386.60,386.05,387.75,387.84,387.94,388.23,388.70,389.25,0,1,0,0,0,0,0,-6,0.056720


In [81]:
#Merge
pd.merge(df,df_new,on = ['TS1','SYMBOL'])

,INSTRUMENT_TOKEN,SYMBOL,OPEN,HIGH,LOW,CLOSE,VOLUME,TS1,TS2,TIMEFRAME_x,P,BC,TC,TIMEFRAME_y,CPR_WIDE,CPR_NARROW,L1,L2,L3,L4,L5,L6,H1,H2,H3,H4,H5,H6,BULLISH,BEARISH,MODERATELY_BULLISH,MODERATELY_BEARISH,NEUTRAL,SIDEWAYS,BREAKOUT,SCORE,CPR
0,3861249,ADANIPORTS,485.00,486.20,483.10,485.10,300003,2021-01-01,2021-01-01 09:30:00,15minute,505.15,505.08,505.22,DAY,0,0,504.52,504.04,503.56,502.11,499.72,496.94,505.48,505.96,506.44,507.89,510.28,513.06,1,0,0,0,0,0,0,3,0.027715
1,3861249,ADANIPORTS,485.10,485.60,482.80,485.50,188231,2021-01-01,2021-01-01 09:45:00,15minute,505.15,505.08,505.22,DAY,0,0,504.52,504.04,503.56,502.11,499.72,496.94,505.48,505.96,506.44,507.89,510.28,513.06,1,0,0,0,0,0,0,3,0.027715
2,3861249,ADANIPORTS,485.45,487.95,484.50,487.55,231754,2021-01-01,2021-01-01 10:00:00,15minute,505.15,505.08,505.22,DAY,0,0,504.52,504.04,503.56,502.11,499.72,496.94,505.48,505.96,506.44,507.89,510.28,513.06,1,0,0,0,0,0,0,3,0.027715
3,3861249,ADANIPORTS,487.50,487.55,486.20,486.95,175421,2021-01-01,2021-01-01 10:15:00,15minute,505.15,505.08,505.22,DAY,0,0,504.52,504.04,503.56,502.11,499.72,496.94,505.48,505.96,506.44,507.89,510.28,513.06,1,0,0,0,0,0,0,3,0.027715
4,3861249,ADANIPORTS,486.95,487.00,486.00,486.75,83198,2021-01-01,2021-01-01 10:30:00,15minute,505.15,505.08,505.22,DAY,0,0,504.52,504.04,503.56,502.11,499.72,496.94,505.48,505.96,506.44,507.89,510.28,513.06,1,0,0,0,0,0,0,3,0.027715
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3445,259849,NIFTYIT,24456.15,24473.00,24448.15,24459.55,0,2021-01-01,2021-01-01 14:30:00,15minute,24461.60,24458.37,24464.82,DAY,0,0,24448.82,24442.49,24436.16,24417.17,24386.03,24349.66,24461.48,24467.81,24474.14,24493.13,24524.27,24560.64,1,0,0,0,0,0,0,1,0.026368
3446,259849,NIFTYIT,24458.15,24486.65,24453.30,24485.60,0,2021-01-01,2021-01-01 14:45:00,15minute,24461.60,24458.37,24464.82,DAY,0,0,24448.82,24442.49,24436.16,24417.17,24386.03,24349.66,24461.48,24467.81,24474.14,24493.13,24524.27,24560.64,1,0,0,0,0,0,0,1,0.026368
3447,259849,NIFTYIT,24482.60,24489.70,24467.55,24470.80,0,2021-01-01,2021-01-01 15:00:00,15minute,24461.60,24458.37,24464.82,DAY,0,0,24448.82,24442.49,24436.16,24417.17,24386.03,24349.66,24461.48,24467.81,24474.14,24493.13,24524.27,24560.64,1,0,0,0,0,0,0,1,0.026368
3448,259849,NIFTYIT,24471.65,24471.65,24435.00,24447.00,0,2021-01-01,2021-01-01 15:15:00,15minute,24461.60,24458.37,24464.82,DAY,0,0,24448.82,24442.49,24436.16,24417.17,24386.03,24349.66,24461.48,24467.81,24474.14,24493.13,24524.27,24560.64,1,0,0,0,0,0,0,1,0.026368


In [166]:
df_final = pd.merge(df,df_new,on = "SYMBOL")

In [79]:
pd.set_option('display.max_columns', None)

In [88]:
df_final.head()

,INSTRUMENT_TOKEN,SYMBOL,OPEN,HIGH,LOW,CLOSE,VOLUME,TS1_x,TS2,TIMEFRAME_x,TS1_y,P,BC,TC,TIMEFRAME_y,CPR_WIDE,CPR_NARROW,L1,L2,L3,L4,L5,L6,H1,H2,H3,H4,H5,H6,BULLISH,BEARISH,MODERATELY_BULLISH,MODERATELY_BEARISH,NEUTRAL,SIDEWAYS,BREAKOUT,SCORE,CPR
0,3861249,ADANIPORTS,311.00,314.70,310.60,312.10,302748,2020-08-03,2020-08-03 09:30:00,15minute,2021-01-01,505.15,505.08,505.22,DAY,0,0,504.52,504.04,503.56,502.11,499.72,496.94,505.48,505.96,506.44,507.89,510.28,513.06,1,0,0,0,0,0,0,3,0.027715
1,3861249,ADANIPORTS,312.35,313.25,310.60,313.05,176071,2020-08-03,2020-08-03 09:45:00,15minute,2021-01-01,505.15,505.08,505.22,DAY,0,0,504.52,504.04,503.56,502.11,499.72,496.94,505.48,505.96,506.44,507.89,510.28,513.06,1,0,0,0,0,0,0,3,0.027715
2,3861249,ADANIPORTS,313.05,313.30,311.50,311.65,66802,2020-08-03,2020-08-03 10:00:00,15minute,2021-01-01,505.15,505.08,505.22,DAY,0,0,504.52,504.04,503.56,502.11,499.72,496.94,505.48,505.96,506.44,507.89,510.28,513.06,1,0,0,0,0,0,0,3,0.027715
3,3861249,ADANIPORTS,311.65,312.45,311.55,312.00,29944,2020-08-03,2020-08-03 10:15:00,15minute,2021-01-01,505.15,505.08,505.22,DAY,0,0,504.52,504.04,503.56,502.11,499.72,496.94,505.48,505.96,506.44,507.89,510.28,513.06,1,0,0,0,0,0,0,3,0.027715
4,3861249,ADANIPORTS,312.10,313.85,311.90,313.55,39762,2020-08-03,2020-08-03 10:30:00,15minute,2021-01-01,505.15,505.08,505.22,DAY,0,0,504.52,504.04,503.56,502.11,499.72,496.94,505.48,505.96,506.44,507.89,510.28,513.06,1,0,0,0,0,0,0,3,0.027715


In [84]:
df_final.dtypes

INSTRUMENT_TOKEN        int64
SYMBOL                 object
OPEN                  float64
HIGH                  float64
LOW                   float64
CLOSE                 float64
VOLUME                  int64
TS1_x                  object
TS2                    object
TIMEFRAME_x            object
TS1_y                  object
P                     float64
BC                    float64
TC                    float64
TIMEFRAME_y            object
CPR_WIDE                int32
CPR_NARROW              int32
L1                    float64
L2                    float64
L3                    float64
L4                    float64
L5                    float64
L6                    float64
H1                    float64
H2                    float64
H3                    float64
H4                    float64
H5                    float64
H6                    float64
BULLISH                 int32
BEARISH                 int32
MODERATELY_BULLISH      int32
MODERATELY_BEARISH      int32
NEUTRAL   

In [83]:
CAMARILLA_VALUES = df_new 
OHLC_15_MINS_PURE_FROM_DATABASE = df 
CAMARILLA_VALUES['CPR'] = abs(CAMARILLA_VALUES.TC - CAMARILLA_VALUES.BC)*100/CAMARILLA_VALUES.P
OHLC_15_MINS_PURE = OHLC_15_MINS_PURE_FROM_DATABASE.loc[(OHLC_15_MINS_PURE_FROM_DATABASE.TS1.dt.date == datetime.date.today()),:]

CAMARILLA_VALUES['H3_LOWER_BAND'] = CAMARILLA_VALUES,H3 - ((CAMARILLA_VALUES.CPR)*0.5)
CAMARILLA_VALUES['H3_UPPER_BAND'] = CAMARILLA_VALUES.H3 + ((CAMARILLA_VALUES.CPR)*0.5)
CAMARILLA_VALUES['H3_UPPER_BAND'] = np.where((CAMARILLA_VALUES.H3_UPPER_BAND) >= (CAMARILLA_VALUES.H3*1.0005),
                                                            (CAMARILLA_VALUES.H3*1.0005),
                                                            (CAMARILLA_VALUES.H3_UPPER_BAND))
CAMARILLA_VALUES['H3_LOWER_BAND'] = np.where((CAMARILLA_VALUES.H3_LOWER_BAND) <= (CAMARILLA_VALUES.H3*0.9995),(CAMARILLA_VALUES.H3*0.9995),
                                                            (CAMARILLA_VALUES.H3_LOWER_BAND))
CAMARILLA_VALUES['H3_MAX_LIMIT'] = CAMARILLA_VALUES.H3 + (0.33 * (CAMARILLA_VALUES.H4-CAMARILLA_VALUES.H3))

CAMARILLA_VALUES['L3_LOWER_BAND'] = CAMARILLA_VALUES.L3 + CAMARILLA_VALUES.CPR
CAMARILLA_VALUES['L3_UPPER_BAND'] = CAMARILLA_VALUES.L3 - CAMARILLA_VALUES.CPR
CAMARILLA_VALUES['L3_UPPER_BAND'] = np.where(
(CAMARILLA_VALUES.L3_UPPER_BAND) >= (CAMARILLA_VALUES.L3*0.9995),(CAMARILLA_VALUES.L3*0.9995))
CAMARILLA_VALUES['L3_LOWER_BAND'] = np.where(
                                                            (CAMARILLA_VALUES.L3_LOWER_BAND) >= (CAMARILLA_VALUES.L3*1.0005),
                                                            (CAMARILLA_VALUES.L3*1.0005),
                                                            (CAMARILLA_VALUES.L3_LOWER_BAND))
CAMARILLA_VALUES['L3_MAX_LIMIT'] = CAMARILLA_VALUES.L3 + (0.33 * (CAMARILLA_VALUES.L4-CAMARILLA_VALUES.L3))

FILTERED_CAMARILLA_VALUES = CAMARILLA_VALUES[['SYMBOL','TS1','TC','CPR','P','BC','H5','H3_MAX_LIMIT','H3_UPPER_BAND','H4','H3_LOWER_BAND','H3','L3_LOWER_BAND','L3','L3_UPPER_BAND','L3_MAX_LIMIT','L4','L5']].round(2)
OHLC_15_MINS_PURE.columns = ['ZERODHA_ID','SYMBOL','OPEN','HIGH','LOW','CLOSE','VOLUME','TS1_15_MINS']
OHLC_15_MINS_PURE = OHLC_15_MINS_PURE.sort_values(['SYMBOL','TS1_15_MINS','ZERODHA_ID'])
OHLC_15_MINS_PURE = OHLC_15_MINS_PURE.groupby(['SYMBOL']).tail(25)
MERGED_DATAFRAME = pd.merge(OHLC_15_MINS_PURE,FILTERED_CAMARILLA_VALUES,on='SYMBOL')

MERGED_DATAFRAME = pd.merge(OHLC_15_MINS_PURE,FILTERED_CAMARILLA_VALUES,on='SYMBOL')

AttributeError: Can only use .dt accessor with datetimelike values

In [89]:
df_final

,INSTRUMENT_TOKEN,SYMBOL,OPEN,HIGH,LOW,CLOSE,VOLUME,TS1_x,TS2,TIMEFRAME_x,TS1_y,P,BC,TC,TIMEFRAME_y,CPR_WIDE,CPR_NARROW,L1,L2,L3,L4,L5,L6,H1,H2,H3,H4,H5,H6,BULLISH,BEARISH,MODERATELY_BULLISH,MODERATELY_BEARISH,NEUTRAL,SIDEWAYS,BREAKOUT,SCORE,CPR
0,3861249,ADANIPORTS,311.00,314.70,310.60,312.10,302748,2020-08-03,2020-08-03 09:30:00,15minute,2021-01-01,505.15,505.08,505.22,DAY,0,0,504.52,504.04,503.56,502.11,499.72,496.94,505.48,505.96,506.44,507.89,510.28,513.06,1,0,0,0,0,0,0,3,0.027715
1,3861249,ADANIPORTS,312.35,313.25,310.60,313.05,176071,2020-08-03,2020-08-03 09:45:00,15minute,2021-01-01,505.15,505.08,505.22,DAY,0,0,504.52,504.04,503.56,502.11,499.72,496.94,505.48,505.96,506.44,507.89,510.28,513.06,1,0,0,0,0,0,0,3,0.027715
2,3861249,ADANIPORTS,313.05,313.30,311.50,311.65,66802,2020-08-03,2020-08-03 10:00:00,15minute,2021-01-01,505.15,505.08,505.22,DAY,0,0,504.52,504.04,503.56,502.11,499.72,496.94,505.48,505.96,506.44,507.89,510.28,513.06,1,0,0,0,0,0,0,3,0.027715
3,3861249,ADANIPORTS,311.65,312.45,311.55,312.00,29944,2020-08-03,2020-08-03 10:15:00,15minute,2021-01-01,505.15,505.08,505.22,DAY,0,0,504.52,504.04,503.56,502.11,499.72,496.94,505.48,505.96,506.44,507.89,510.28,513.06,1,0,0,0,0,0,0,3,0.027715
4,3861249,ADANIPORTS,312.10,313.85,311.90,313.55,39762,2020-08-03,2020-08-03 10:30:00,15minute,2021-01-01,505.15,505.08,505.22,DAY,0,0,504.52,504.04,503.56,502.11,499.72,496.94,505.48,505.96,506.44,507.89,510.28,513.06,1,0,0,0,0,0,0,3,0.027715
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
366247,259849,NIFTYIT,24456.15,24473.00,24448.15,24459.55,0,2021-01-01,2021-01-01 14:30:00,15minute,2021-01-01,24461.60,24458.37,24464.82,DAY,0,0,24448.82,24442.49,24436.16,24417.17,24386.03,24349.66,24461.48,24467.81,24474.14,24493.13,24524.27,24560.64,1,0,0,0,0,0,0,1,0.026368
366248,259849,NIFTYIT,24458.15,24486.65,24453.30,24485.60,0,2021-01-01,2021-01-01 14:45:00,15minute,2021-01-01,24461.60,24458.37,24464.82,DAY,0,0,24448.82,24442.49,24436.16,24417.17,24386.03,24349.66,24461.48,24467.81,24474.14,24493.13,24524.27,24560.64,1,0,0,0,0,0,0,1,0.026368
366249,259849,NIFTYIT,24482.60,24489.70,24467.55,24470.80,0,2021-01-01,2021-01-01 15:00:00,15minute,2021-01-01,24461.60,24458.37,24464.82,DAY,0,0,24448.82,24442.49,24436.16,24417.17,24386.03,24349.66,24461.48,24467.81,24474.14,24493.13,24524.27,24560.64,1,0,0,0,0,0,0,1,0.026368
366250,259849,NIFTYIT,24471.65,24471.65,24435.00,24447.00,0,2021-01-01,2021-01-01 15:15:00,15minute,2021-01-01,24461.60,24458.37,24464.82,DAY,0,0,24448.82,24442.49,24436.16,24417.17,24386.03,24349.66,24461.48,24467.81,24474.14,24493.13,24524.27,24560.64,1,0,0,0,0,0,0,1,0.026368


In [224]:
def Signal(df_final):
    signal = []
    flag = -1
    
    for i in df_final.iterrows():
        if( df_final['OPEN'][i] <= df_final['H2'][i] and df_final['CLOSE'][i] >= df_final['H2'][i] ):
            if(flag != 1):
                signal.append(df_final['CLOSE'][i])
                flag = 1
        
        else:
            signal.append(np.nan)
            
    return(signal)
    

SyntaxError: invalid syntax (<ipython-input-224-b95ba3922da1>, line 11)

In [142]:
len(df_final)

366252

In [160]:
Signal(df_final)

[nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan

In [193]:
df_final[df_final["H4"].between(df_final['OPEN'], df_final['CLOSE'], inclusive = True)] == True

,INSTRUMENT_TOKEN,SYMBOL,OPEN,HIGH,LOW,CLOSE,VOLUME,TS1_x,TS2,TIMEFRAME_x,TS1_y,P,BC,TC,TIMEFRAME_y,CPR_WIDE,CPR_NARROW,L1,L2,L3,L4,L5,L6,H1,H2,H3,H4,H5,H6,BULLISH,BEARISH,MODERATELY_BULLISH,MODERATELY_BEARISH,NEUTRAL,SIDEWAYS,BREAKOUT,SCORE,CPR,Signal
5291,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,True,False,False,False,False,False,False,False,False
5293,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,True,False,False,False,False,False,False,False,False
7184,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False
7187,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False
7213,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
363543,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,True,False,False,False,False,False,False,False,False
363561,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,True,False,False,False,False,False,False,False,False
363569,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,True,False,False,False,False,False,False,False,False
363584,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,True,False,False,False,False,False,False,False,False


In [209]:
signal = []
flag = -1
for i in range(len(df_final)):
    if( df_final['CLOSE'][i] <= df_final['H2'][i] ) :
            if(flag != 1):
                #signal.append(df_final['CLOSE'][i])
                df_final['signal'] = 1
                flag = 1
        
    else:
            #signal.append(np.nan)
            df_final['signal'] = 0

In [167]:
#df_final['OPEN'] <= df_final['H2'] and df_final['CLOSE'] >= df_final['H2']
#[df_final["H2"].between(df_final['HIGH'], df_final['LOW'], inclusive = True)] == True
df_final['Signal'] = 0

In [223]:
#Applyying lambda functions
#df_final.Signal = df_final.Sinal.apply(lambda x:if_statemnt if conditon else else_statement)

#df_final.signal.apply(lambda x: df_final.signal = 1 if df_final['HIGH'] <= df_final['H2'] else df_final.signal = 0)
df.apply(lambda x: df if df_final.signal = 1 if df_final['HIGH'] <= df_final['H2'] else df_f, axis=1) 

SyntaxError: invalid syntax (<ipython-input-223-351926f12095>, line 3)

In [257]:
#Logic 3
def signal(df_final):
    signal = []
    #sigPriceSell = []
    flag = -1  
    
    for i in range(len(df_final)):
        
        if (df_final['LOW'][i] <= df_final['H2'][i]) and (df_final['HIGH'][i] >= df_final['H2'][i]):
            
            if flag != 1:
                
                signal.append(1)
                flag = 1
                
            else:
                
                pass
                #sigPriceSell.append(np.nan)
                
        '''elif (df_final['HIGH'][i] <= df_final['H2'][i]) and (df_final['LOW'][i] >= df_final['H2'][i]):
            
            if flag != 0:
                
                signal.append(1)
                
                flag = 0
            else:
                
                signal.append(np.nan)
               
        else:
            
            signal.append(np.nan)
            '''

In [258]:
sig = signal(df_final)


#df['Buy_Signal_Price'] = buy_sell[0]
#df['Sell_Signal_Price'] = buy_sell[1]

In [254]:
df_final['signal']  = 0

In [255]:
df_final

,INSTRUMENT_TOKEN,SYMBOL,OPEN,HIGH,LOW,CLOSE,VOLUME,TS1_x,TS2,TIMEFRAME_x,TS1_y,P,BC,TC,TIMEFRAME_y,CPR_WIDE,CPR_NARROW,L1,L2,L3,L4,L5,L6,H1,H2,H3,H4,H5,H6,BULLISH,BEARISH,MODERATELY_BULLISH,MODERATELY_BEARISH,NEUTRAL,SIDEWAYS,BREAKOUT,SCORE,CPR,signal
0,3861249,ADANIPORTS,311.00,314.70,310.60,312.10,302748,2020-08-03,2020-08-03 09:30:00,15minute,2021-01-01,505.15,505.08,505.22,DAY,0,0,504.52,504.04,503.56,502.11,499.72,496.94,505.48,505.96,506.44,507.89,510.28,513.06,1,0,0,0,0,0,0,3,0.027715,0
1,3861249,ADANIPORTS,312.35,313.25,310.60,313.05,176071,2020-08-03,2020-08-03 09:45:00,15minute,2021-01-01,505.15,505.08,505.22,DAY,0,0,504.52,504.04,503.56,502.11,499.72,496.94,505.48,505.96,506.44,507.89,510.28,513.06,1,0,0,0,0,0,0,3,0.027715,0
2,3861249,ADANIPORTS,313.05,313.30,311.50,311.65,66802,2020-08-03,2020-08-03 10:00:00,15minute,2021-01-01,505.15,505.08,505.22,DAY,0,0,504.52,504.04,503.56,502.11,499.72,496.94,505.48,505.96,506.44,507.89,510.28,513.06,1,0,0,0,0,0,0,3,0.027715,0
3,3861249,ADANIPORTS,311.65,312.45,311.55,312.00,29944,2020-08-03,2020-08-03 10:15:00,15minute,2021-01-01,505.15,505.08,505.22,DAY,0,0,504.52,504.04,503.56,502.11,499.72,496.94,505.48,505.96,506.44,507.89,510.28,513.06,1,0,0,0,0,0,0,3,0.027715,0
4,3861249,ADANIPORTS,312.10,313.85,311.90,313.55,39762,2020-08-03,2020-08-03 10:30:00,15minute,2021-01-01,505.15,505.08,505.22,DAY,0,0,504.52,504.04,503.56,502.11,499.72,496.94,505.48,505.96,506.44,507.89,510.28,513.06,1,0,0,0,0,0,0,3,0.027715,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
366247,259849,NIFTYIT,24456.15,24473.00,24448.15,24459.55,0,2021-01-01,2021-01-01 14:30:00,15minute,2021-01-01,24461.60,24458.37,24464.82,DAY,0,0,24448.82,24442.49,24436.16,24417.17,24386.03,24349.66,24461.48,24467.81,24474.14,24493.13,24524.27,24560.64,1,0,0,0,0,0,0,1,0.026368,0
366248,259849,NIFTYIT,24458.15,24486.65,24453.30,24485.60,0,2021-01-01,2021-01-01 14:45:00,15minute,2021-01-01,24461.60,24458.37,24464.82,DAY,0,0,24448.82,24442.49,24436.16,24417.17,24386.03,24349.66,24461.48,24467.81,24474.14,24493.13,24524.27,24560.64,1,0,0,0,0,0,0,1,0.026368,0
366249,259849,NIFTYIT,24482.60,24489.70,24467.55,24470.80,0,2021-01-01,2021-01-01 15:00:00,15minute,2021-01-01,24461.60,24458.37,24464.82,DAY,0,0,24448.82,24442.49,24436.16,24417.17,24386.03,24349.66,24461.48,24467.81,24474.14,24493.13,24524.27,24560.64,1,0,0,0,0,0,0,1,0.026368,0
366250,259849,NIFTYIT,24471.65,24471.65,24435.00,24447.00,0,2021-01-01,2021-01-01 15:15:00,15minute,2021-01-01,24461.60,24458.37,24464.82,DAY,0,0,24448.82,24442.49,24436.16,24417.17,24386.03,24349.66,24461.48,24467.81,24474.14,24493.13,24524.27,24560.64,1,0,0,0,0,0,0,1,0.026368,0


In [259]:
def signal(data):
    signal_yes = []
    signal_no = []
    flag = -1  
    
    for i in range(len(data)):
        
        if (data['LOW'][i] <= data['H2'][i]) and (data['HIGH'][i] >= data['H2'][i]):
            
            if flag != 1:
                
                signal_yes.append(1)
                signal_no.append(np.nan)
                flag = 1
            else:
                
                signal_yes.append(np.nan)
                signal_no.append(np.nan)
                
        elif (data['HIGH'][i] <= data['H2'][i]) and (data['LOW'][i] >= data['H2'][i]):
            
            if flag != 0:
                
                signal_yes.append(np.nan)
                signal_no.append(1)
                flag = 0
            else:
                
                signal_yes.append(np.nan)
                signal__no.append(np.nan)
        else:
            
            signal_yes.append(np.nan)
            signal_no.append(np.nan)
            
    return(signal_yes,signal_no)

In [261]:
signal = signal(df_final)

In [263]:
df_final['Breakout'] = signal[0]
df_final['Breakin'] = signal[1]

In [ ]:
def RSI_Option(rsi_options):
    df=rsi_options.copy()
    df['TS1'] = pd.to_datetime(df['TS1'])
    df=df.sort_values(['SYMBOL','TS1'])
    df = IND_RSI(df,period=14).dropna()
    df['level_1'] = df.groupby(['SYMBOL']).cumcount()
    df_Spot_OHLC_ATR_15M = df.groupby(['SYMBOL']).apply(lambda x:
                                                                  pd.DataFrame(np.vstack(TA.ATR(x.HIGH.values,
                                                                                                x.LOW.values,
                                                                                                x.CLOSE.values,
                                                                                                timeperiod=14)),
                                                                               columns=['ATR'])
                                                                  ).reset_index()


    df1=pd.merge(df,
                 df_Spot_OHLC_ATR_15M,
              how = 'left',
              left_on = ['SYMBOL','level_1'],
              right_on = ['SYMBOL','level_1'])

    df2=df1.groupby(['SYMBOL']).apply(lambda x :
                                        pd.DataFrame({
                                 'SYMBOL' : x.SYMBOL,
                                 'OPEN' : x.OPEN,
                                 'HIGH' : x.HIGH,
                                 'LOW' : x.LOW,
                                 'CLOSE' : x.CLOSE,
                                 'TS1' : x.TS1,
                                 'VOL' : x.VOL,
                                 'RSI' : x.RSI,
                                 'ATR' : x.ATR,
                                 'PREVIOUS_RSI' : x.RSI.values[-2],
                                 }))

    df2=df2.groupby(['SYMBOL']).last().reset_index()

    df2['SIGNAL']=np.where(((df2.RSI>60) & (df2.PREVIOUS_RSI<60)),'CE',np.where(((df2.RSI<40) & (df2.PREVIOUS_RSI>40)),'PE',0))
    SCAN_RSI=pd.read_sql_table('SCAN_RSI', engine).drop(['id'],axis=1)

    SCAN_RSI=SCAN_RSI[SCAN_RSI.TIMEFRAME=='60MIN']

    merged_Rsi=pd.merge(df2, SCAN_RSI, how='left'
                     , left_on=['SYMBOL'], right_on=['SYMBOL'],suffixes=('_30', '_60'))

    merged_Rsi=merged_Rsi[['SYMBOL','ATR','HIGH_30','LOW_30','OPEN_30','PREVIOUS_RSI','RSI_30','TS1_30','VOL','SIGNAL','RSI_60']]
    merged_Rsi.columns=['SYMBOL','ATR','HIGH','LOW','OPEN','PREVIOUS_RSI','RSI_30','TS1','VOL','SIGNAL','RSI_60']

    merged_Rsi['CURATED_SIGNAL']=np.where(((merged_Rsi.SIGNAL=='CE') & (merged_Rsi.RSI_60>50)),1,np.where(((merged_Rsi.SIGNAL=='PE') & (merged_Rsi.RSI_60<50)),1,0))
    universe_symbol_options= pd.read_sql_table('UNIVERSE_SYMBOL_OPTIONS', engine).drop(['id'],axis=1)   
    merged_Rsi=merged_Rsi[merged_Rsi.SYMBOL.isin(universe_symbol_options.SYMBOL)]
    merged_Rsi=merged_Rsi[merged_Rsi['TS1'].dt.strftime("%H:%M")<="14:15"]
    return merged_Rsi

In [ ]:
def Scan_Trident (df_OHLC_Data,df_Pivot,TF=0):
        
    df_Trident = df_OHLC_Data.groupby('SYMBOL').apply(lambda x:
                                                      pd.Series({'EMA_13P_HIGH_2':
                                                                 TA.EMA(x.HIGH.values,timeperiod = 13)[-2:].round(2),
                                                                 'EMA_13P_LOW_2':
                                                                 TA.EMA(x.LOW.values,timeperiod = 13)[-2:].round(2),
                                                                 'EMA_5P_CLOSE_2':
                                                                 TA.EMA(x.CLOSE.values,timeperiod = 5)[-2:].round(2),
                                                                 'CLOSE':
                                                                 x.CLOSE.values[-1],
                                                                 'ATR':
                                                                 x.ATR.values[-1],
                                                                 'ATR_PCT':
                                                                 x.ATR.values[-1]*100/x.CLOSE.values[-1],
                                                                 'REX':
                                                                 x.REX.values[-1],
                                                                 'RIX':
                                                                 x.RIX.values[-1],
                                                                 'PIVOT_HIGH':
                                                                 np.max(x.HIGH.values[-6:-1]),
                                                                 'PIVOT_LOW':
                                                                 np.min(x.LOW.values[-6:-1]),
                                                                 'CANDLE_COLOR':
                                                                 np.where(x.CLOSE.values[-1]>=x.OPEN.values[-1],
                                                                          1,-1
                                                                         ),
                                                                 'CF':
                                                                 (abs(x.CLOSE.values[-1] - x.OPEN.values[-1])/
                                                                  abs(x.HIGH.values[-1] - x.LOW.values[-1]))*100,
                                                                 'CUS':
                                                                 ((x.HIGH.values[-1]-max(x.OPEN.values[-1],x.CLOSE.values[-1]))/
                                                                  (abs(x.CLOSE.values[-1] - x.OPEN.values[-1]))),
                                                                 'CLS':
                                                                 ((x.LOW.values[-1]-min(x.OPEN.values[-1],x.CLOSE.values[-1]))/
                                                                  (abs(x.CLOSE.values[-1] - x.OPEN.values[-1]))),
                                                                 'RANGE_BO':
                                                                 ((x.HIGH.values[-1] - x.LOW.values[-1])>
                                                                  max((x.HIGH.values[-2] - x.LOW.values[-2]),
                                                                      (x.HIGH.values[-3] - x.LOW.values[-3]),
                                                                      (x.HIGH.values[-4] - x.LOW.values[-4]),
                                                                      (x.HIGH.values[-5] - x.LOW.values[-5]),
                                                                      (x.HIGH.values[-6] - x.LOW.values[-6])
                                                                     )),
                                                                 
                                                                })).reset_index()
    

    df_Trident[['EMA_13P_HIGH_2','EMA_13P_HIGH_1']] = pd.DataFrame(df_Trident.EMA_13P_HIGH_2.values.tolist(),
                                                                     index= df_Trident.index)
    df_Trident[['EMA_13P_LOW_2','EMA_13P_LOW_1']] = pd.DataFrame(df_Trident.EMA_13P_LOW_2.values.tolist(),
                                                                     index= df_Trident.index)
    df_Trident[['EMA_5P_CLOSE_2','EMA_5P_CLOSE_1']] = pd.DataFrame(df_Trident.EMA_5P_CLOSE_2.values.tolist(),
                                                                     index= df_Trident.index)
    
    df_Trident = pd.merge(df_Trident,
                          df_Pivot[['SYMBOL','R1','S1','PV_H','PV_L']],
                          how = 'left',
                          on = 'SYMBOL') 

    #df_Trident
    df_Trident['ALERT'] = 0 
    df_Trident['STOPLOSS'] = 0.0
    df_Trident['TARGET'] = 0.0
    df_Trident['TARGET_2'] = 0.0
    df_Trident['TARGET_3'] = 0.0
    df_Trident['Re_Ri'] = 0.0
    df_Trident['SL_TRAIL'] = 0.0
    

    Alert_Up = ((df_Trident['CLOSE'] > df_Trident['EMA_13P_HIGH_1'])&
                (df_Trident['CLOSE'] > df_Trident['R1']))
    Alert_Down = ((df_Trident['CLOSE'] < df_Trident['EMA_13P_LOW_1'])&
                  (df_Trident['CLOSE'] < df_Trident['S1']))
    
    #ATR_TGT = 2
    #ATR_SL = .3
    
    if (TF == 0):
        
        df_Trident.loc[Alert_Up,
                         ['ALERT','STOPLOSS','TARGET']] = pd.DataFrame({'ALERT':1,
                                                                        #'STOPLOSS':df_Trident.EMA_13P_LOW_1,
                                                                        'STOPLOSS':df_Trident.CLOSE-(df_Trident.RIX*
                                                                                                     df_Trident.ATR),
                                                                        'TARGET':(df_Trident.CLOSE+(df_Trident.REX*
                                                                                                    df_Trident.ATR))},
                                                                       index=df_Trident.index)
        df_Trident.loc[Alert_Down,
                         ['ALERT','STOPLOSS','TARGET']] = pd.DataFrame({'ALERT':-1,
                                                                        #'STOPLOSS':df_Trident.EMA_13P_HIGH_1,
                                                                        'STOPLOSS':df_Trident.CLOSE+(df_Trident.RIX*
                                                                                                     df_Trident.ATR),
                                                                        'TARGET':(df_Trident.CLOSE-(df_Trident.REX*
                                                                                                    df_Trident.ATR))},
                                                                 index=df_Trident.index)
        
    if (TF == 2):
        
        df_Trident.loc[Alert_Up,
                         ['ALERT','STOPLOSS','TARGET','TARGET_2','TARGET_3']] = pd.DataFrame({'ALERT':1,
                                                                        'STOPLOSS':df_Trident.CLOSE*(1-0.0105),#MAE=1.05%
                                                                        'TARGET':(df_Trident.CLOSE+(df_Trident.REX*
                                                                                                    df_Trident.ATR)),
                                                                        'TARGET_2':(df_Trident.CLOSE*(1+.0045)), #MFE1 = 0.45%
                                                                        'TARGET_3':(df_Trident.CLOSE*(1+.012)) #MFE2 = 1.2%
                                                                       },
                                                                       index=df_Trident.index)
        df_Trident.loc[Alert_Down,
                         ['ALERT','STOPLOSS','TARGET','TARGET_2','TARGET_3']] = pd.DataFrame({'ALERT':-1,
                                                                        'STOPLOSS':df_Trident.CLOSE*(1+0.0105), #MAE=1.05%
                                                                        'TARGET':(df_Trident.CLOSE-(df_Trident.REX*
                                                                                                    df_Trident.ATR)),
                                                                        'TARGET_2':(df_Trident.CLOSE*(1-.0045)), #MFE1 = 0.45%
                                                                        'TARGET_3':(df_Trident.CLOSE*(1-.012)) #MFE2 = 1.2%
                                                                       },
                                                                 index=df_Trident.index)
        
    if (TF == 3):
        
        df_Trident.loc[Alert_Up,
                         ['ALERT','STOPLOSS','TARGET','TARGET_2','TARGET_3']] = pd.DataFrame({'ALERT':1,
                                                                        'STOPLOSS':df_Trident.CLOSE*(1-0.012),#MAE=1.2%
                                                                        'TARGET':(df_Trident.CLOSE+(df_Trident.REX*
                                                                                                    df_Trident.ATR)),
                                                                        'TARGET_2':(df_Trident.CLOSE*(1+.006)), #MFE1 = 0.6%
                                                                        'TARGET_3':(df_Trident.CLOSE*(1+.012)) #MFE2 = 1.2%
                                                                       },
                                                                       index=df_Trident.index)
        df_Trident.loc[Alert_Up,
                         ['ALERT','STOPLOSS','TARGET','TARGET_2','TARGET_3']] = pd.DataFrame({'ALERT':-1,
                                                                        'STOPLOSS':df_Trident.CLOSE*(1+0.0105), #MAE=1.05%
                                                                        'TARGET':(df_Trident.CLOSE-(df_Trident.REX*
                                                                                                    df_Trident.ATR)),
                                                                        'TARGET_2':(df_Trident.CLOSE*(1-.006)), #MFE1 = 0.6%
                                                                        'TARGET_3':(df_Trident.CLOSE*(1-.012)) #MFE2 = 1.2%
                                                                       },
                                                                 index=df_Trident.index)
    
    #df_Trident.loc[(Alert_Down|Alert_Up),['Re_Ri']] = pd.DataFrame({'Re_Ri':
    #                                                                (abs(df_Trident.TARGET-df_Trident.CLOSE)/
    #                                                                 abs(df_Trident.STOPLOSS-df_Trident.CLOSE))},
    #                                                               index=df_Trident.index)

    
    df_Trident['SIGNAL'] = 0
    df_Trident['CURATED_SIGNAL'] = 0
    df_Trident['ENTRY'] = 0.0
    
    if(TF==0):
   
        Signal = (     (
                        (df_Trident.ALERT == 1)&
                        (df_Trident['EMA_5P_CLOSE_1'] > df_Trident['EMA_13P_HIGH_1'])&
                        (df_Trident['EMA_5P_CLOSE_2'] <= df_Trident['EMA_13P_HIGH_2'])&
                        (df_Trident['CANDLE_COLOR'] == 1)

                       )|
                       (
                        (df_Trident.ALERT == -1)&
                        (df_Trident['EMA_5P_CLOSE_1'] < df_Trident['EMA_13P_LOW_1'])&
                        (df_Trident['EMA_5P_CLOSE_2'] >= df_Trident['EMA_13P_LOW_2'])&
                        (df_Trident['CANDLE_COLOR'] == -1)
                       )
                      )
    
    if(TF==2):
   
        Signal = (     (
                        (df_Trident.ALERT == 1)&
                        (df_Trident['EMA_5P_CLOSE_1'] > df_Trident['EMA_13P_HIGH_1'])&
                        (df_Trident['EMA_5P_CLOSE_2'] <= df_Trident['EMA_13P_HIGH_2'])&
                        (df_Trident['CANDLE_COLOR'] == 1)&
                        (df_Trident.ATR_PCT>0.9)&
                        (df_Trident.ATR_PCT<=1.2)

                       )|
                       (
                        (df_Trident.ALERT == -1)&
                        (df_Trident['EMA_5P_CLOSE_1'] < df_Trident['EMA_13P_LOW_1'])&
                        (df_Trident['EMA_5P_CLOSE_2'] >= df_Trident['EMA_13P_LOW_2'])&
                        (df_Trident['CANDLE_COLOR'] == -1)&
                        (df_Trident.ATR_PCT>0.9)&
                        (df_Trident.ATR_PCT<=1.2)
                       )
                      )
    
    if(TF==3):
   
        Signal = (     (
                        (df_Trident.ALERT == 1)&
                        (df_Trident['EMA_5P_CLOSE_1'] > df_Trident['EMA_13P_HIGH_1'])&
                        (df_Trident['EMA_5P_CLOSE_2'] <= df_Trident['EMA_13P_HIGH_2'])&
                        (df_Trident['CANDLE_COLOR'] == 1)&
                        (df_Trident.ATR_PCT>0.9)&
                        (df_Trident.ATR_PCT<=1.35)

                       )|
                       (
                        (df_Trident.ALERT == -1)&
                        (df_Trident['EMA_5P_CLOSE_1'] < df_Trident['EMA_13P_LOW_1'])&
                        (df_Trident['EMA_5P_CLOSE_2'] >= df_Trident['EMA_13P_LOW_2'])&
                        (df_Trident['CANDLE_COLOR'] == -1)&
                        (df_Trident.ATR_PCT>0.9)&
                        (df_Trident.ATR_PCT<=1.35)
                       )
                      )
    
    df_Trident.loc[Signal,
                   ['SIGNAL','ENTRY']] = pd.DataFrame({'SIGNAL':1,
                                                       'ENTRY':df_Trident.CLOSE},
                                                      index=df_Trident.index)

    
    Curated_Signal = ((
        
                    (df_Trident.SIGNAL == 1)&
                    (df_Trident['CLOSE'] > df_Trident['PV_H'])&
                    (df_Trident['CLOSE'] > df_Trident['PIVOT_HIGH'])
                   )|
                   (
                   (df_Trident.SIGNAL == 1)&
                   (df_Trident['CLOSE'] < df_Trident['PV_L'])&
                   (df_Trident['CLOSE'] < df_Trident['PIVOT_LOW'])
                   )
                  )


    
    df_Trident.loc[Curated_Signal,
                     ['CURATED_SIGNAL']] = pd.DataFrame({'CURATED_SIGNAL':1
                                                         },
                                                        index=df_Trident.index)

    return df_Trident

In [332]:
df_final.head()

,INSTRUMENT_TOKEN,SYMBOL,OPEN,HIGH,LOW,CLOSE,VOLUME,TS1_x,TS2,TIMEFRAME_x,TS1_y,P,BC,TC,TIMEFRAME_y,CPR_WIDE,CPR_NARROW,L1,L2,L3,L4,L5,L6,H1,H2,H3,H4,H5,H6,BULLISH,BEARISH,MODERATELY_BULLISH,MODERATELY_BEARISH,NEUTRAL,SIDEWAYS,BREAKOUT,SCORE,CPR,signal
0,3861249,ADANIPORTS,311.00,314.70,310.60,312.10,302748,2020-08-03,2020-08-03 09:30:00,15minute,2021-01-01,505.15,505.08,505.22,DAY,0,0,504.52,504.04,503.56,502.11,499.72,496.94,505.48,505.96,506.44,507.89,510.28,513.06,1,0,0,0,0,0,0,3,0.027715,0
1,3861249,ADANIPORTS,312.35,313.25,310.60,313.05,176071,2020-08-03,2020-08-03 09:45:00,15minute,2021-01-01,505.15,505.08,505.22,DAY,0,0,504.52,504.04,503.56,502.11,499.72,496.94,505.48,505.96,506.44,507.89,510.28,513.06,1,0,0,0,0,0,0,3,0.027715,0
2,3861249,ADANIPORTS,313.05,313.30,311.50,311.65,66802,2020-08-03,2020-08-03 10:00:00,15minute,2021-01-01,505.15,505.08,505.22,DAY,0,0,504.52,504.04,503.56,502.11,499.72,496.94,505.48,505.96,506.44,507.89,510.28,513.06,1,0,0,0,0,0,0,3,0.027715,0
3,3861249,ADANIPORTS,311.65,312.45,311.55,312.00,29944,2020-08-03,2020-08-03 10:15:00,15minute,2021-01-01,505.15,505.08,505.22,DAY,0,0,504.52,504.04,503.56,502.11,499.72,496.94,505.48,505.96,506.44,507.89,510.28,513.06,1,0,0,0,0,0,0,3,0.027715,0
4,3861249,ADANIPORTS,312.10,313.85,311.90,313.55,39762,2020-08-03,2020-08-03 10:30:00,15minute,2021-01-01,505.15,505.08,505.22,DAY,0,0,504.52,504.04,503.56,502.11,499.72,496.94,505.48,505.96,506.44,507.89,510.28,513.06,1,0,0,0,0,0,0,3,0.027715,0


In [346]:
df_final = np.where(((df_final.LOW <= df_final.H4) & (df_final.HIGH >= df_final.H4)),1,np.where(((df_final.HIGH <= df_final.H4) & (df_final.LOW >= df_final.H4)),1,0))

array([0, 0, 0, ..., 0, 0, 1])

In [333]:
df_final.signal.value_counts()

0    360073
1      6179
Name: signal, dtype: int64

In [334]:
df_final[df_final['signal'] == 1]

,INSTRUMENT_TOKEN,SYMBOL,OPEN,HIGH,LOW,CLOSE,VOLUME,TS1_x,TS2,TIMEFRAME_x,TS1_y,P,BC,TC,TIMEFRAME_y,CPR_WIDE,CPR_NARROW,L1,L2,L3,L4,L5,L6,H1,H2,H3,H4,H5,H6,BULLISH,BEARISH,MODERATELY_BULLISH,MODERATELY_BEARISH,NEUTRAL,SIDEWAYS,BREAKOUT,SCORE,CPR,signal
5291,60417,ASIANPAINT,2770.00,2780.00,2769.55,2778.50,98229,2021-01-01,2021-01-01 11:30:00,15minute,2021-01-01,2770.82,2770.46,2771.18,DAY,0,0,2768.97,2767.84,2766.70,2763.31,2757.73,2751.21,2771.23,2772.36,2773.50,2776.89,2782.47,2788.99,0,1,0,0,0,0,0,-5,0.025985,1
5292,60417,ASIANPAINT,2778.50,2779.40,2773.00,2773.50,36600,2021-01-01,2021-01-01 11:45:00,15minute,2021-01-01,2770.82,2770.46,2771.18,DAY,0,0,2768.97,2767.84,2766.70,2763.31,2757.73,2751.21,2771.23,2772.36,2773.50,2776.89,2782.47,2788.99,0,1,0,0,0,0,0,-5,0.025985,1
5293,60417,ASIANPAINT,2773.50,2781.85,2773.25,2776.95,40690,2021-01-01,2021-01-01 12:00:00,15minute,2021-01-01,2770.82,2770.46,2771.18,DAY,0,0,2768.97,2767.84,2766.70,2763.31,2757.73,2751.21,2771.23,2772.36,2773.50,2776.89,2782.47,2788.99,0,1,0,0,0,0,0,-5,0.025985,1
5294,60417,ASIANPAINT,2776.95,2779.90,2776.00,2778.50,18676,2021-01-01,2021-01-01 12:15:00,15minute,2021-01-01,2770.82,2770.46,2771.18,DAY,0,0,2768.97,2767.84,2766.70,2763.31,2757.73,2751.21,2771.23,2772.36,2773.50,2776.89,2782.47,2788.99,0,1,0,0,0,0,0,-5,0.025985,1
5306,60417,ASIANPAINT,2784.95,2785.00,2774.10,2776.75,114364,2021-01-01,2021-01-01 15:15:00,15minute,2021-01-01,2770.82,2770.46,2771.18,DAY,0,0,2768.97,2767.84,2766.70,2763.31,2757.73,2751.21,2771.23,2772.36,2773.50,2776.89,2782.47,2788.99,0,1,0,0,0,0,0,-5,0.025985,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
363591,260105,BANKNIFTY,31253.70,31284.15,31238.05,31275.50,0,2021-01-01,2021-01-01 14:00:00,15minute,2021-01-01,31224.35,31217.72,31230.98,DAY,0,0,31203.95,31196.81,31189.66,31168.23,31133.10,31092.08,31218.25,31225.39,31232.54,31253.97,31289.10,31330.12,0,1,0,0,0,0,0,-3,0.042467,1
363596,260105,BANKNIFTY,31277.55,31277.55,31208.95,31227.05,0,2021-01-01,2021-01-01 15:15:00,15minute,2021-01-01,31224.35,31217.72,31230.98,DAY,0,0,31203.95,31196.81,31189.66,31168.23,31133.10,31092.08,31218.25,31225.39,31232.54,31253.97,31289.10,31330.12,0,1,0,0,0,0,0,-3,0.042467,1
363597,260105,BANKNIFTY,31229.45,31269.95,31192.00,31211.10,0,2021-01-01,2021-01-01 15:30:00,15minute,2021-01-01,31224.35,31217.72,31230.98,DAY,0,0,31203.95,31196.81,31189.66,31168.23,31133.10,31092.08,31218.25,31225.39,31232.54,31253.97,31289.10,31330.12,0,1,0,0,0,0,0,-3,0.042467,1
366233,259849,NIFTYIT,24452.55,24499.45,24450.85,24469.35,0,2021-01-01,2021-01-01 11:00:00,15minute,2021-01-01,24461.60,24458.37,24464.82,DAY,0,0,24448.82,24442.49,24436.16,24417.17,24386.03,24349.66,24461.48,24467.81,24474.14,24493.13,24524.27,24560.64,1,0,0,0,0,0,0,1,0.026368,1


In [341]:
df_plot = pd.DataFrame()
df_plot['Open'] = df_final.OPEN
df_plot['High'] = df_final.HIGH
df_plot['Low'] = df_final.LOW
df_plot['Close'] = df_final.CLOSE
df_plot['SIGNAL'] = df_final.signal
df_plot['Date'] = df_final.TS1_x

In [338]:
df_plot.info()

<class 'pandas.core.frame.DataFrame'>
DatetimeIndex: 366252 entries, 2020-08-03 to 2021-01-01
Data columns (total 5 columns):
 #   Column  Non-Null Count   Dtype  
---  ------  --------------   -----  
 0   Open    366252 non-null  float64
 1   High    366252 non-null  float64
 2   Low     366252 non-null  float64
 3   Close   366252 non-null  float64
 4   SIGNAL  366252 non-null  int32  
dtypes: float64(4), int32(1)
memory usage: 15.4 MB


In [342]:
df_plot.Date = pd.to_datetime(df_plot['Date'])

In [343]:
#df_plot.reset_index(inplace = True)
df_plot = df_plot.set_index('Date')

In [344]:
df_plot.head()

,Open,High,Low,Close,SIGNAL
Date,,,,,
2020-08-03,311.00,314.70,310.60,312.10,0
2020-08-03,312.35,313.25,310.60,313.05,0
2020-08-03,313.05,313.30,311.50,311.65,0
2020-08-03,311.65,312.45,311.55,312.00,0
2020-08-03,312.10,313.85,311.90,313.55,0
